# w261 Final Project - Clickthrough Rate Prediction


[Your team number (from the spreadsheet)]   
[Your team names]   
Summer 2019, section [Your section numbers>]   

## Table of Contents

* __Section 1__ - Question Formulation
* __Section 2__ - Algorithm Explanation
* __Section 3__ - EDA & Challenges
* __Section 4__ - Algorithm Implementation
* __Section 5__ - Course Concepts

# __Section 1__ - Question Formulation

# __Section 2__ - Algorithm Explanation

# __Section 3__ - EDA & Challenges

# __Section 4__ - Algorithm Implementation

# __Section 5__ - Course Concepts

### Setup and Initiate Spark

In [1]:
import re
import ast
import time
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import Row

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [4]:
# start Spark Session (RUN THIS CELL AS IS)
from pyspark.sql import SparkSession
app_name = "hw3_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

In [5]:
# Spark configuration Information (RUN THIS CELL AS IS)
sc.getConf().getAll()

[('spark.app.id', 'local-1564320575509'),
 ('spark.driver.port', '43701'),
 ('spark.app.name', 'hw3_notebook'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', 'docker.w261')]

### Setup Path and read in raw data files

In [6]:
raw_test = PWD + '/data/test.txt'
raw_train = PWD + '/data/train.txt'

In [7]:
# raw_test = 'gs://deb-261/FinalProj/data/test.txt'
# raw_train = 'gs://deb-261/FinalProj/data/test.txt'

In [8]:
rawTestRDD = sc.textFile(raw_test)
rawTrainRDD = sc.textFile(raw_train)

### Check Row Counts

In [9]:
rawTestRDD.count()
#6042135

6042135

In [9]:
rawTrainRDD.count()
#45840617

45840617

### Parse Row Into Readable formats

In [10]:
def parse_raw_row(row):
    '''
    for each row in the raw data,  outputs the following tuple:
        - (label, features_vector)
        where features_vector is comma separated string of features
        features with no value are replaced by empty string ''
    '''
    row_values = row.split('\t')
    new_values = [value if value != '' else "''" for value in row_values[1:]]
    output_string = ",".join(new_values)
    return (row_values[0], output_string)

### Testing Parse Function

In [11]:
rawTrainRDD.take(2)

['0\t1\t1\t5\t0\t1382\t4\t15\t2\t181\t1\t2\t\t2\t68fd1e64\t80e26c9b\tfb936136\t7b4723c4\t25c83c98\t7e0ccccf\tde7995b8\t1f89b562\ta73ee510\ta8cd5504\tb2cb9c98\t37c9c164\t2824a5f6\t1adce6ef\t8ba8b39a\t891b62e7\te5ba7672\tf54016b9\t21ddcdc9\tb1252a9d\t07b5194c\t\t3a171ecb\tc5c50484\te8b83407\t9727dd16',
 '0\t2\t0\t44\t1\t102\t8\t2\t2\t4\t1\t1\t\t4\t68fd1e64\tf0cf0024\t6f67f7e5\t41274cd7\t25c83c98\tfe6b92e5\t922afcc0\t0b153874\ta73ee510\t2b53e5fb\t4f1b46f3\t623049e6\td7020589\tb28479f6\te6c5b5cd\tc92f3b61\t07c540c4\tb04e4670\t21ddcdc9\t5840adea\t60f6221e\t\t3a171ecb\t43f13e8b\te8b83407\t731c3655']

In [12]:
example = rawTrainRDD.take(5)
exampleRDD=sc.parallelize(example)
exampleRDD.collect()

['0\t1\t1\t5\t0\t1382\t4\t15\t2\t181\t1\t2\t\t2\t68fd1e64\t80e26c9b\tfb936136\t7b4723c4\t25c83c98\t7e0ccccf\tde7995b8\t1f89b562\ta73ee510\ta8cd5504\tb2cb9c98\t37c9c164\t2824a5f6\t1adce6ef\t8ba8b39a\t891b62e7\te5ba7672\tf54016b9\t21ddcdc9\tb1252a9d\t07b5194c\t\t3a171ecb\tc5c50484\te8b83407\t9727dd16',
 '0\t2\t0\t44\t1\t102\t8\t2\t2\t4\t1\t1\t\t4\t68fd1e64\tf0cf0024\t6f67f7e5\t41274cd7\t25c83c98\tfe6b92e5\t922afcc0\t0b153874\ta73ee510\t2b53e5fb\t4f1b46f3\t623049e6\td7020589\tb28479f6\te6c5b5cd\tc92f3b61\t07c540c4\tb04e4670\t21ddcdc9\t5840adea\t60f6221e\t\t3a171ecb\t43f13e8b\te8b83407\t731c3655',
 '0\t2\t0\t1\t14\t767\t89\t4\t2\t245\t1\t3\t3\t45\t287e684f\t0a519c5c\t02cf9876\tc18be181\t25c83c98\t7e0ccccf\tc78204a1\t0b153874\ta73ee510\t3b08e48b\t5f5e6091\t8fe001f4\taa655a2f\t07d13a8f\t6dc710ed\t36103458\t8efede7f\t3412118d\t\t\te587c466\tad3062eb\t3a171ecb\t3b183c5c\t\t',
 '0\t\t893\t\t\t4392\t\t0\t0\t0\t\t0\t\t\t68fd1e64\t2c16a946\ta9a87e68\t2e17d6f6\t25c83c98\tfe6b92e5\t2e8a689b\t0b15387

In [13]:
exampleRDD.map(parse_raw_row).collect()

[('0',
  "1,1,5,0,1382,4,15,2,181,1,2,'',2,68fd1e64,80e26c9b,fb936136,7b4723c4,25c83c98,7e0ccccf,de7995b8,1f89b562,a73ee510,a8cd5504,b2cb9c98,37c9c164,2824a5f6,1adce6ef,8ba8b39a,891b62e7,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,'',3a171ecb,c5c50484,e8b83407,9727dd16"),
 ('0',
  "2,0,44,1,102,8,2,2,4,1,1,'',4,68fd1e64,f0cf0024,6f67f7e5,41274cd7,25c83c98,fe6b92e5,922afcc0,0b153874,a73ee510,2b53e5fb,4f1b46f3,623049e6,d7020589,b28479f6,e6c5b5cd,c92f3b61,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,'',3a171ecb,43f13e8b,e8b83407,731c3655"),
 ('0',
  "2,0,1,14,767,89,4,2,245,1,3,3,45,287e684f,0a519c5c,02cf9876,c18be181,25c83c98,7e0ccccf,c78204a1,0b153874,a73ee510,3b08e48b,5f5e6091,8fe001f4,aa655a2f,07d13a8f,6dc710ed,36103458,8efede7f,3412118d,'','',e587c466,ad3062eb,3a171ecb,3b183c5c,'',''"),
 ('0',
  "'',893,'','',4392,'',0,0,0,'',0,'','',68fd1e64,2c16a946,a9a87e68,2e17d6f6,25c83c98,fe6b92e5,2e8a689b,0b153874,a73ee510,efea433b,e51ddf94,a30567ca,3516f6e6,07d13a8f,18231224,52b8680f,1e88c74

### Creating Parsed Data Sets from Raw Training and Test data

In [14]:
start = time.time()
parsedTrainRDD = rawTrainRDD.map(parse_raw_row).cache()
parsedTestRDD = rawTestRDD.map(parse_raw_row).cache()
print(f"\n... parsed raw data rows in {time.time() - start} seconds")


... parsed raw data rows in 0.09914064407348633 seconds


### Split Training data into Train, Dev, Test subsets

In [15]:
trainRDD, devRDD, testRDD = parsedTrainRDD.randomSplit([0.8,0.1, 0.1], seed = 2018)

In [16]:
trainRDD.take(5)

[('0',
  "2,0,44,1,102,8,2,2,4,1,1,'',4,68fd1e64,f0cf0024,6f67f7e5,41274cd7,25c83c98,fe6b92e5,922afcc0,0b153874,a73ee510,2b53e5fb,4f1b46f3,623049e6,d7020589,b28479f6,e6c5b5cd,c92f3b61,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,'',3a171ecb,43f13e8b,e8b83407,731c3655"),
 ('0',
  "2,0,1,14,767,89,4,2,245,1,3,3,45,287e684f,0a519c5c,02cf9876,c18be181,25c83c98,7e0ccccf,c78204a1,0b153874,a73ee510,3b08e48b,5f5e6091,8fe001f4,aa655a2f,07d13a8f,6dc710ed,36103458,8efede7f,3412118d,'','',e587c466,ad3062eb,3a171ecb,3b183c5c,'',''"),
 ('0',
  "'',893,'','',4392,'',0,0,0,'',0,'','',68fd1e64,2c16a946,a9a87e68,2e17d6f6,25c83c98,fe6b92e5,2e8a689b,0b153874,a73ee510,efea433b,e51ddf94,a30567ca,3516f6e6,07d13a8f,18231224,52b8680f,1e88c74f,74ef3502,'','',6b3a5ca6,'',3a171ecb,9117a34a,'',''"),
 ('0',
  "3,-1,'',0,2,0,3,0,0,1,1,'',0,8cf07265,ae46a29d,c81688bb,f922efad,25c83c98,13718bbd,ad9fa255,0b153874,a73ee510,5282c137,e5d8af57,66a76a26,f06c53ac,1adce6ef,8ff4b403,01adbab4,1e88c74f,26b3c7a7,'','',21c9516a,''

In [16]:
devRDD.take(5)

[('0',
  "1,1,5,0,1382,4,15,2,181,1,2,'',2,68fd1e64,80e26c9b,fb936136,7b4723c4,25c83c98,7e0ccccf,de7995b8,1f89b562,a73ee510,a8cd5504,b2cb9c98,37c9c164,2824a5f6,1adce6ef,8ba8b39a,891b62e7,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,'',3a171ecb,c5c50484,e8b83407,9727dd16"),
 ('0',
  "0,1,80,0,1848,287,1,4,46,0,1,'',4,05db9164,09e68b86,13b87f72,13a91973,25c83c98,7e0ccccf,cc5ed2f1,0b153874,a73ee510,3b08e48b,081c279a,d25f00b6,9f16a973,07d13a8f,36721ddc,1746d357,d4bb7bd8,5aed7436,a153cea2,a458ea53,dd37e0d1,'',32c7478e,c70a58f2,e8b83407,af7ece63"),
 ('0',
  "4,0,55,8,859,13,4,12,13,1,1,'',8,05db9164,e5fb1af3,4b644986,5dbf0cc5,25c83c98,'',cc5ed2f1,0b153874,a73ee510,3b08e48b,facf05cc,6d89b6a5,9f16a973,cfef1c29,1e744fde,2bd32e5c,776ce399,13145934,21ddcdc9,a458ea53,1419c3fc,'',32c7478e,8ecc176a,e8b83407,a70a038a"),
 ('0',
  "8,0,15,20,115,24,8,23,24,2,2,'',20,5a9ed9b0,c66fca21,78171040,373c404a,25c83c98,'',8ff6f5af,0b153874,a73ee510,5ba575e7,b5a9f90e,6766a7f0,949ea585,1adce6ef,8736735c,59974c9c,

In [17]:
testRDD.take(5)

[('1',
  "0,-1,'','',1465,0,17,0,4,0,4,'','',241546e0,38a947a1,fa673455,6a14f9b9,25c83c98,fe6b92e5,1c86e0eb,1f89b562,a73ee510,e7ba2569,755e4a50,208d9687,5978055e,07d13a8f,5182f694,f8b34416,e5ba7672,e5f8f18f,'','',f3ddd519,'',32c7478e,b34f3128,'',''"),
 ('0',
  "0,-1,'','',4894,20,1,7,20,0,1,'','',05db9164,4c2bc594,d032c263,c18be181,43b19349,7e0ccccf,7f52e00f,0b153874,a73ee510,48a4f593,bca79aeb,dfbb09fb,5218d824,8ceecbc8,7ac43a46,84898b2a,07c540c4,bc48b783,'','',0014c32a,c9d4222a,3a171ecb,3b183c5c,'',''"),
 ('0',
  "0,190,'','',1624,6,29,6,74,0,9,'','',68fd1e64,c41a84c8,0a266224,759c4a2e,25c83c98,'',804d2f11,0b153874,a73ee510,2860ede1,1aa6cf31,99dfd83a,3b03d76e,b28479f6,e3eb97c7,62b5674b,e5ba7672,5911fc7e,'','',28ee216d,'',32c7478e,590b856f,'',''"),
 ('0',
  "39,8,42,32,27,33,39,24,32,1,1,'',32,05db9164,73a46ff0,844ce0a4,9bb11257,4cf72387,7e0ccccf,ff3f3dda,0fb392dd,a73ee510,3b08e48b,e2217f93,1d0b8187,da9ee8bd,1adce6ef,d57668e2,4372eb4b,e5ba7672,da507f45,21ddcdc9,5840adea,ecb5cd6f,'',32c

In [18]:
#cell for sanity check, takes a while to run (30 min)
print(trainRDD.count()) #36671527
print(devRDD.count()) #4584772
print(testRDD.count()) #4584318

36671527
4584772
4584318


In [19]:
36671527+4584772+4584318

45840617

In [18]:
#toyTrain = trainRDD.takeSample(False, 5)

In [19]:
## Take toy data set for just 100 samples

#toyTrain = trainRDD.takeSample(False, 5)
toyTrain = trainRDD.take(10)
toytrainRDD = sc.parallelize(toyTrain)

In [20]:
toytrainRDD.take(2)

[('0',
  "2,0,44,1,102,8,2,2,4,1,1,'',4,68fd1e64,f0cf0024,6f67f7e5,41274cd7,25c83c98,fe6b92e5,922afcc0,0b153874,a73ee510,2b53e5fb,4f1b46f3,623049e6,d7020589,b28479f6,e6c5b5cd,c92f3b61,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,'',3a171ecb,43f13e8b,e8b83407,731c3655"),
 ('0',
  "2,0,1,14,767,89,4,2,245,1,3,3,45,287e684f,0a519c5c,02cf9876,c18be181,25c83c98,7e0ccccf,c78204a1,0b153874,a73ee510,3b08e48b,5f5e6091,8fe001f4,aa655a2f,07d13a8f,6dc710ed,36103458,8efede7f,3412118d,'','',e587c466,ad3062eb,3a171ecb,3b183c5c,'',''")]

## Hash Encode the Feature Vector ##

In [21]:
def helper_for_hash(row):
    '''
    input = row : tuple(label, One string holding all the features separated by comma)
    output = tuple(label, list of feature values)
    '''
    feature = row[1]
    lst = feature.split(',')
    return (row[0],[lst[i+1] for i in range(len(lst)-1)])

In [31]:
import hashlib
def hashing(row):
    '''
    for each row in the raw data, use hashing function
    to take care of the categorical variables
        input = row : 
    '''
    hash_buckets = 2 ** 26
    new_value = []
    for value in row[1]:
        new_value.append(int(int(hashlib.md5(str(value).encode('utf-8')).hexdigest(), 16) % hash_buckets))
    return((row[0], new_value))


In [32]:
toyTrain = trainRDD.take(100)
toytrainRDD = sc.parallelize(toyTrain)

In [33]:
start = time.time()
toytrainRDD1 = toytrainRDD.map(lambda x:helper_for_hash(x)).cache()
toytrainRDD_hashed = toytrainRDD1.map(lambda x: hashing(x)).cache()
print(f"\n... toy data hashed in {time.time() - start} seconds")


... toy data hashed in 0.08796501159667969 seconds


In [34]:
toytrainRDD_hashed.take(2)

[('0',
  [25650394,
   53419718,
   58033307,
   61656670,
   25043821,
   1345068,
   1345068,
   21107244,
   58033307,
   58033307,
   47312746,
   21107244,
   45827163,
   12131770,
   24281840,
   61207377,
   41509161,
   32253967,
   24569265,
   12320039,
   56793155,
   50517202,
   42139888,
   40939996,
   66604463,
   55650035,
   3936003,
   34843665,
   26503176,
   14002281,
   10463523,
   4006755,
   47847845,
   47312746,
   21565591,
   47786363,
   39920753,
   11662870]),
 ('0',
  [25650394,
   58033307,
   33545302,
   66914630,
   9500470,
   21107244,
   1345068,
   40205719,
   58033307,
   44546803,
   44546803,
   27801999,
   40438944,
   45396177,
   7044192,
   65064450,
   41509161,
   61861415,
   61718064,
   12320039,
   56793155,
   15214302,
   10622483,
   44172847,
   36579957,
   39027416,
   16219057,
   37339987,
   65140142,
   54441174,
   47312746,
   47312746,
   25581665,
   886458,
   21565591,
   20857345,
   47312746,
   47312746])]